In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from lightgbm import LGBMClassifier

# 1. 데이터 로드 (파일 경로를 수정해서 사용하세요)
df = pd.read_csv('./machine_failure.csv')

df.rename(columns={
    'Air temperature [K]': 'Air_temp_K',
    'Process temperature [K]': 'Process_temp_K',
    'Rotational speed [rpm]': 'Rotational_speed_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Tool_wear_min'
}, inplace=True)

# 2. 특성 엔지니어링 (물리적 관계를 변수로 추가)
def engineering(df):
    # 온도 차이: HDF 고장 예측에 핵심
    df['Temp_Diff'] = df['Process_temp_K'] - df['Air_temp_K']
    # 파워(토크 * 속도): PWF, OSF 고장 예측에 핵심
    df['Power'] = df['Torque_Nm'] * df['Rotational_speed_rpm']
    # 마모 가중치
    df['Wear_Torque'] = df['Tool_wear_min'] * df['Torque_Nm']
    return df

df = engineering(df)

# 3. 데이터 전처리 (카테고리 변수 변환 및 불필요 컬럼 제거)
# Type(L, M, H)을 숫자로 변환
df['Type'] = df['Type'].map({'L': 0, 'M': 1, 'H': 2})

# 학습에 사용할 특성(X)과 타겟(y) 분리
features = ['Type', 'Air_temp_K', 'Process_temp_K', 
            'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min', 
            'Temp_Diff', 'Power', 'Wear_Torque']

# A. 고장 여부 예측용 타겟 (Binary)
X = df[features]
y_bin = df['Machine failure']

# B. 고장 유형 분류용 타겟 (Multi-class)
# 여러 고장 컬럼을 하나의 레이블로 통합 (0: 정상, 1: TWF, 2: HDF, 3: PWF, 4: OSF, 5: RNF)
def get_multiclass_target(row):
    if row['TWF'] == 1: return 1
    if row['HDF'] == 1: return 2
    if row['PWF'] == 1: return 3
    if row['OSF'] == 1: return 4
    if row['RNF'] == 1: return 5
    return 0

y_multi = df.apply(get_multiclass_target, axis=1)

# 4. 고장 여부(이진 분류) 모델 학습
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.2, stratify=y_bin, random_state=42)

model_bin = LGBMClassifier(n_estimators=100, class_weight='balanced', random_state=42)
model_bin.fit(X_train, y_train)

# 5. 결과 확인
print("### 고장 여부 예측 결과 (Binary Classification) ###")
y_pred = model_bin.predict(X_test)
print(classification_report(y_test, y_pred))

# 6. 고장 종류(다중 분류) 모델 학습
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X, y_multi, test_size=0.2, stratify=y_multi, random_state=42)

model_multi = LGBMClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model_multi.fit(X_train_m, y_train_m)

print("\n### 고장 종류 예측 결과 (Multi-class Classification) ###")
y_pred_m = model_multi.predict(X_test_m)
print(classification_report(y_test_m, y_pred_m))

[LightGBM] [Info] Number of positive: 271, number of negative: 7729
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001543 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1518
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
### 고장 여부 예측 결과 (Binary Classification) ###
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1932
           1       0.79      0.84      0.81        68

    accuracy                           0.99      2000
   macro avg       0.89      0.92      0.90      2000
weighted avg       0.99      0.99      0.99      2000

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of te

d:\Users\admin\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Users\admin\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Users\admin\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape

In [14]:
import pandas as pd
import numpy as np

# [중요] 컬럼명 변경을 가장 먼저 확실하게 수행
df.rename(columns={
    'Air temperature [K]': 'Air_temp_K',
    'Process temperature [K]': 'Process_temp_K',
    'Rotational speed [rpm]': 'Rotational_speed_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Tool_wear_min'
}, inplace=True)

# 1. RNF 제거 전 데이터 확인
print(f"원본 데이터 개수: {len(df)}")

# RNF가 존재하는지 확인 후 제거
if 'RNF' in df.columns:
    df_clean = df[df['RNF'] == 0].copy()
else:
    df_clean = df.copy()

print(f"RNF 제거 후 데이터 개수: {len(df_clean)}")

# 2. 특성 생성 (컬럼명이 바뀐 것을 반영)
df_clean['Temp_Diff'] = df_clean['Process_temp_K'] - df_clean['Air_temp_K']
df_clean['Power'] = df_clean['Torque_Nm'] * df_clean['Rotational_speed_rpm']
df_clean['Wear_Torque_Ratio'] = df_clean['Tool_wear_min'] * df_clean['Torque_Nm']
df_clean['Wear_Level'] = np.where(df_clean['Tool_wear_min'] > 180, 2, 
                                  np.where(df_clean['Tool_wear_min'] > 100, 1, 0))

# 3. 결측치 처리 전후 확인
# dropna를 하기 전에 어떤 컬럼에 결측치가 있는지 확인
print("컬럼별 결측치 개수:\n", df_clean.isnull().sum())

# 결측치가 있는 행만 제거 (Target_Class 생성 후 수행)
def get_target(row):
    if row['TWF'] == 1: return 1
    if row['HDF'] == 1: return 2
    if row['PWF'] == 1: return 3
    if row['OSF'] == 1: return 4
    return 0

df_clean['Target_Class'] = df_clean.apply(get_target, axis=1)

features = ['Type', 'Air_temp_K', 'Process_temp_K', 'Rotational_speed_rpm', 
            'Torque_Nm', 'Tool_wear_min', 'Temp_Diff', 'Power', 
            'Wear_Torque_Ratio', 'Wear_Level']

# 실제로 데이터가 존재하는지 최종 확인
df_final = df_clean.dropna(subset=features + ['Target_Class'])
print(f"최종 학습 데이터 개수: {len(df_final)}")

if len(df_final) == 0:
    print("경고: 최종 데이터가 0개입니다. dropna 조건을 확인하세요.")
else:
    # 데이터가 있을 때만 분할 및 학습 진행
    X = df_final[features]
    X = pd.get_dummies(X, columns=['Type'], drop_first=True) # Type이 문자열일 경우 대비
    y = df_final['Target_Class']

# 4. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 5. SMOTE를 이용한 데이터 증식 (TWF 등 소수 클래스 보강)
# 각 고장 유형을 최소 400개씩으로 늘려 모델이 충분히 학습하게 함
sm = SMOTE(sampling_strategy={1: 400, 2: 400, 3: 400, 4: 400}, k_neighbors=3, random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# 6. 모델 학습 (TWF 미세 패턴 포착을 위한 파라미터 튜닝)
model_final = LGBMClassifier(
    n_estimators=500,        # 학습 횟수 증가
    learning_rate=0.03,      # 조금 더 세밀하게 학습
    num_leaves=63,           # 모델의 복잡도 조절
    min_child_samples=5,     # 적은 양의 데이터 패턴도 놓치지 않도록 설정
    class_weight='balanced', # 클래스 불균형 자동 보정
    random_state=42
)

model_final.fit(X_train_res, y_train_res)

# 7. 결과 확인
y_pred = model_final.predict(X_test)
print("### RNF 제거 및 특성 강화 후 결과 ###")
print(classification_report(y_test, y_pred))

원본 데이터 개수: 9981
RNF 제거 후 데이터 개수: 9981
컬럼별 결측치 개수:
 UDI                        0
Product ID                 0
Type                    9981
Air_temp_K                 0
Process_temp_K             0
Rotational_speed_rpm       0
Torque_Nm                  0
Tool_wear_min              0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
Temp_Diff                  0
Power                      0
Wear_Torque                0
Wear_Torque_Ratio          0
Target_Class               0
Wear_Level                 0
dtype: int64
최종 학습 데이터 개수: 0
경고: 최종 데이터가 0개입니다. dropna 조건을 확인하세요.


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.